## Chapter 4 - Parameters and metrics



### Functions that return values



When you run NumPy's `sqrt`, it returns a number you can assign to a
variable:



In [1]:
from numpy import sqrt

root_2 = sqrt(2)
print(root_2)

1.4142135623730951

When you run `State`, it returns a new `State` object:



In [1]:
from modsim import State, show

bikeshare = State(leap=10,city=2)
print(show(bikeshare))

state
leap     10
city      2

When you run `step`, it updates a `State` object but it does not return a
value.

Create a function `add_five` that takes a parameter `x` (any number), and
directly returns `x + 5`. Run it with `x=3` and print the result.



In [1]:
def add_five(x):
    return x + 5

print(add_five(3))

8

What if you stored `x + 5` in a variable `result` and returned nothing?
Could you still print `result` outside `add_five` somehow?



In [1]:
def add_five(x):
    global result
    result = x + 5
    return

print(result)

8

Create a version of `run_simulation` that creates a `State` object, runs a
simulation, and then returns the `State` object (instead of plotting
it):

1.  As parameters, use only `p1`, `p2`, `num_steps`
2.  Define a `state` with all four state variables: 10 leap, 2 city
    bikes, no unhappy customers
3.  Iterate `step(state,p1,p2)` over `num_steps`
4.  `return` `state`



In [1]:
# def run simulation
def run_simulation(p1,p2,num_steps):
    state = State(leap=10,city=2,leap_empty=0,city_empty=0)

    for i in range(num_steps):
        step(state,p1,p2)
    return state

Call `run_simulation` for `p1=0.3`, `p2=0.2` and `num_steps=60`, store the
result in `final_state`, and `show` this final state:



In [1]:
final_state = run_simulation(0.3,0.2,60)
print(show(final_state))

state
leap            0
city           12
leap_empty      1
city_empty      0

The result is a `State` object that represents the final state of the
system, including the *metrics* we'll use to evaluate the performance of
the system.

Print the final unhappy customer values using an f-string. Print
unhappy customers for each location on one line.



In [1]:
print(f'Unhappy customers at LEAP: {final_state.leap_empty}\nUnhappy customers at city: {final_state.city_empty}')

Unhappy customers at LEAP: 1
Unhappy customers at city: 0

Our model parameters are `p1=0.3`, `p2=0.2`, `num_steps`, `leap=10` and `city=2`.



### Loops and arrays



Setting the loop limit in a `for` loop with `range` only works with
integers:

    for i in range(num_steps):
        step(state,p1,p2)

To get a sequence of non-integer values, use NumPy's `linspace`:



In [1]:
from numpy import linspace

p1_array = linspace(from=0,to=1,5)  
print(p1_array)

Check out the `help` for `linspace` - which arguments are mandatory? What
kind of arguments are they? Run the function with minimal arguments:



In [1]:
print(linspace(1,100)) # 50 numbers between 1 and 100
print(len(linspace(1,100))) # check num

#+begin_example
[  1.           3.02040816   5.04081633   7.06122449   9.08163265
  11.10204082  13.12244898  15.14285714  17.16326531  19.18367347
  21.20408163  23.2244898   25.24489796  27.26530612  29.28571429
  31.30612245  33.32653061  35.34693878  37.36734694  39.3877551
  41.40816327  43.42857143  45.44897959  47.46938776  49.48979592
  51.51020408  53.53061224  55.55102041  57.57142857  59.59183673
  61.6122449   63.63265306  65.65306122  67.67346939  69.69387755
  71.71428571  73.73469388  75.75510204  77.7755102   79.79591837
  81.81632653  83.83673469  85.85714286  87.87755102  89.89795918
  91.91836735  93.93877551  95.95918367  97.97959184 100.        ]
50
#+end_example

What `type` of data structure is `linspace`?



In [1]:
print(type(linspace(1,100)))

<class 'numpy.ndarray'>

An array is a container for a sequence of numbers, and you can now
loop directly over the array `p1_array`:



In [1]:
for p1 in p1_array:
    print(p1)

0.0
0.25
0.5
0.75
1.0

### Sweeping parameters



We can use known parameter values `p1` and `p2` to predict system behavior
(e.g. how many bikes will be at LEAP after one hour).

We can also use a range of parameters to analyze the system and design
alternatives, scenarios like adding more bikes or more stations.

Example: if we know that `p2` is about 0.2 but we don't have information
about `p1`, we can run simulations for a range of `p1` values - sweep
through them - and see how the results vary. We could then compare the
results with the actual situation and deduce the value of `p1`.

Experiment:

1.  Create a parameter array `p1_array` of six values `p1` $\in [0,0.6]$.
2.  Set `p2` to 0.2 and `num_steps` to 60
3.  Sweep with `p1` through `p1_array`
4.  In each iteration, run a simulation and print the number of unhappy
    customers at LEAP (the location for `p1`)



In [1]:
# parameter values
p1_array = linspace(0, 0.6, 6)
p2 = 0.2
num_steps = 60

# parameter sweep
for p1 in p1_array:
    final_state = run_simulation(p1, p2, num_steps)
    print(f'p1: {p1} Unhappy customers: {final_state.leap_empty}')

p1: 0.0 Unhappy customers: 0
p1: 0.12 Unhappy customers: 0
p1: 0.24 Unhappy customers: 0
p1: 0.36 Unhappy customers: 0
p1: 0.48 Unhappy customers: 2
p1: 0.6 Unhappy customers: 11

Let's plot the results. To do this, create an empty `SweepSeries` object
and instead of printing the final state variable, store it in the
series (aka vector) `sweep`:



In [1]:
from modsim import SweepSeries
sweep = SweepSeries()
p1_array = linspace(0, 0.6, 31)
p2 = 0.2
num_steps = 60
for p1 in p1_array:
    final_state = run_simulation(p1,p2,num_steps)
    sweep[p1] = final_state.leap_empty

Remember that the `Series` objects are vectors whose elements have an ID
(in this case the probability `p1`) and a value (the performance
metric). The `SweepSeries` maps from each value of `p1` to the
corresponding number of unhappy customers.

Plot `sweep` using `plot`:



In [1]:
sweep.plot(label='LEAP',color='C1')  # TimeSeries was plotting w/'C0'
decorate(title='LEAP-city bikeshare',
         xlabel='Customer rate at LEAP (p1 in customers/min)',
         ylabel='Number of unhappy customers at LEAP')

What have we learnt?

-   When the arrival rate at LEAP is low, there are plenty of bikes and
    no unhappy customers.
-   As the arrival rate increases, we are likely to run out of bikes and
    the number of unhappy customers increases accordingly (though not
    linearly - why not?).



### Incremental model development



Incremental development helps to minimize the pain of debugging
(though we'll switch into debugging mode after fall break):

1.  Always start with a working program.
2.  Make one small, testable change at a time.
3.  Run the program and see if the change worked.

Problems with this approach:

-   You may have to create extra *scaffolding* code to get visible output.
-   In the beginning, it is not obvious how to add code to solve more
    complex problems. Mapping the problem out mathematically might help.



### Summary



This chapter introduced functions that return values, which we used to
write a version of run<sub>simulation</sub> that returns a State object with the
final state of the system. It also introduced `linspace`, which we used
to create a NumPy array, and `SweepSeries`, which we used to store the
results of a parameter `sweep`. We used a parameter `sweep` to explore the
relationship between one of the parameters, `p1`, and the number of
unhappy customers, which is a metric that quantifies how well (or
badly) the system works. In the exercises, you’ll have a chance to
sweep other parameters and compute other metrics.  In the next lesson,
we’ll move on to a new problem, modeling and predicting world
*population growth*.



### Exercises

